<a href="https://colab.research.google.com/github/howard-ops/20251207/blob/main/20251211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import json
import urllib.request as req
import bs4 as bs

def download_meta(url):
  # url = "https://www.ptt.cc/bbs/Beauty/M.1758273800.A.03F.html"
  resp = req.urlopen(url)
  content = resp.read()
  html = bs.BeautifulSoup(content)

  metas = html.find_all("span", {"class":"article-meta-value"})
  uid = metas[0]
  board = metas[1]
  title = metas[2]
  posttime = metas[3]


  uid_text = uid.get_text().strip()
  board_text = board.get_text().strip()
  title_text = title.get_text().strip()
  posttime_text = posttime.get_text().strip()

  pushes = html.find_all("div", {"class":"push"})
  push_list = []
  for p in pushes:
      trans = {"推":1, "→":0, "噓":-1}
      pushtag = p.find("span", {"class":"push-tag"})
      pushuid = p.find("span", {"class":"push-userid"})
      pushcontent = p.find("span", {"class":"push-content"})
      pushipdatetime = p.find("span", {"class":"push-ipdatetime"})

      pushtag_text = pushtag.get_text().strip()
      pushtag_score = trans[pushtag_text]
      pushuid_text = pushuid.get_text().strip()
      pushcontent_text = pushcontent.get_text().strip().replace(": ", "")
      pushipdatetime_text = pushipdatetime.get_text().strip()

      push_meta = {
          "score": pushtag_score,
          "uid": pushuid_text,
          "content": pushcontent_text,
          "ipdatetime": pushipdatetime_text
      }
      push_list.append(push_meta)

  # extract: 我都放最後做(因為刪了就刪了)
  maincontent = html.find("div", {"id":"main-content"})
  targets = html.find_all("div", {"class":"article-metaline"})
  for t in targets:
      t.extract()
  targets = html.find_all("div", {"class":"article-metaline-right"})
  for t in targets:
      t.extract()
  targets = html.find_all("div", {"class":"push"})
  for t in targets:
      t.extract()
  maincontent_text = maincontent.get_text()

  # userid/看板名稱/標題/時間  推虛文: 推需>/userid/內容/ipdatetime
  row = {
      "uid": uid_text,
      "board": board_text,
      "title": title_text,
      "time": posttime_text,
      "content": maincontent_text,
      "pushes": push_list
  }

  dn = url.split("/")[-1]
  fn = dn + "/" + "metadata.json"
  print("Save to:", fn)
  # 如果資料夾不存在, 就make起來
  if not os.path.exists(dn):
      os.makedirs(dn)

  with open(fn, "w", encoding="utf-8") as f:
      json.dump(row, f, ensure_ascii=False, indent=4)

In [38]:
def download_imgs(url):
  # url = "https://www.ptt.cc/bbs/Beauty/M.1758273800.A.03F.html"
  resp = req.urlopen(url)
  content = resp.read()
  html = bs.BeautifulSoup(content)

  allow_subs = ["jpg", "jpeg", "png", "gif", "webp"]
  links = html.find_all("a")
  for l in links:
    l_herf = l["href"]
    l_subname = l_herf.split(".")[-1]
    if l_subname in allow_subs:
      print("donload:", l_herf)

      r = req.Request(l_herf)
      r.add_header("user-agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36")
      resq = req.urlopen(r)
      content = resq.read()

      dn = url.split("/")[-1]
      fn = dn + "/" + l_herf.split("/")[-1]
      print("Save to:", fn)
      # 如果資料夾不存在, 就make起來
      if not os.path.exists(dn):
        os.makedirs(dn)
      with open(fn, "wb") as f:
          f.write(content)
    else:
      print("pass:", l_herf)

In [ ]:
url = "https://www.ptt.cc/bbs/Beauty/index3946.html"
# 走過每個超連結(拿包他的區塊)
# 拿出href, 完整網址->定義好的兩個函式

resp = req.urlopen(url)
content = resp.read()
html = bs.BeautifulSoup(content)

links = html.find_all("div", {"class":"title"})
for l in links:
  print(l)
  #l_href = l["href"]
  #print(l_herf)

In [ ]:
url = "https://www.ptt.cc/bbs/Beauty/index3999.html"
# 走過每個超連結(拿包他的區塊)
# 拿出href, 完整網址->定義好的兩個函式
resp = req.urlopen(url)
content = resp.read()
html = bs.BeautifulSoup(content)

post_divs = html.find_all("div", {"class":"title"})
for div in post_divs:
    post_link = div.find("a")
    # == None: 裡面沒有a, 被刪文了
    if not post_link == None:
        post_link_href = "https://www.ptt.cc" + post_link["href"]
        download_meta(post_link_href)
        download_imgs(post_link_href)
        print(post_link_href)
    else:
        print("被刪文了!!")

In [ ]:
import json
import glob
import pandas as pd

table = []
for fn in glob.glob("*/metadata.json"):
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
        row = json.loads(content)
        table.append(row)
df = pd.DataFrame(table)
df